# 하이퍼파라미터 튜닝

<img src="https://github.com/ElaYJ/supplement/assets/153154981/7b953cab-04e5-4c7e-835c-3c7cb9cf227f" width="67%" height="67%"></br>

</br>

- HyperParameter란? 기계 학습 모델 훈련을 관리하는 데 사용하는 외부 구성 변수

	- 하이퍼파라미터는 모델을 훈련하기 전에 수동으로 설정된다.
	
	- 하이퍼파라미터는 파라미터와는 다르다.</br>
		데이터 과학자에 의해 설정되는 것이 아닌 학습 프로세스 중에 자동으로 파생되는 내부 파라미터이다.

	- 하이퍼파라미터의 예로는 신경망의 노드 및 계층 수와 **의사 결정 트리의 분기 수**가 있다.
	- 하이퍼파라미터는 모델 아키텍처, 학습 속도 및 모델 복잡성과 같은 주요 기능을 결정한다.
	- 올바른 하이퍼파라미터 세트를 선택하는 것은 모델 성능과 정확성 측면에서 중요하다.
	- 하이퍼파라미터가 가장 잘 작동하는 설정된 규칙이나 최적 또는 기본값은 없다.
	- 최적의 하이퍼파라미터 세트를 찾으려면 실험을 해야 한다.
	- 이러한 활동을 하이퍼파라미터 튜닝 또는 하이퍼파라미터 최적화라고 한다.

</br>

- Hyperparameter Tuning이란?

	기계 학습 모델을 훈련 중인 경우, 각 데이터 세트와 모델에는 일종의 변수인 다양한 하이퍼파라미터 세트가 필요하다.</br>
	이를 결정하는 유일한 방법은 여러 실험을 통해 하이퍼파라미터 세트를 선택하고 모델을 통해 실행하는 것이다.</br>
	이를 하이퍼파라미터 튜닝이라고 한다.

#### --▶ 튜닝 대상

- 결정나무(Decision Tree)에서 우리가 튜닝해 볼만한 하이퍼파라미터는 max_depth 하나 뿐이다.

- 간단하게 반복문으로 max_depth를 바꿔가며 테스트해볼 수 있다.

- 앞으로를 생각해서 보다 간편하고 유용한 방법을 생각해보자.

In [1]:
import pandas as pd

red_wine = pd.read_csv("../dataset/winequality-red.csv", sep=';')
white_wine = pd.read_csv("../dataset/winequality-white.csv", sep=';')

red_wine['color'] = 1.
white_wine['color'] = 0.

wine = pd.concat([red_wine, white_wine])

wine['taste'] = [1. if grade > 5 else 0. for grade in wine['quality']]

X = wine.drop(['taste','quality'], axis=1) #--> column drop
y = wine['taste']

## 🔰 GridSearchCV

- Grid Search Cross Validation

- Grid Search는 가장 기본적인 하이퍼파라미터 최적화 방법이다.

- Grid Search는 가능한 모든 조합의 하이퍼파라미터로 훈련시켜서 최적의 조합을 찾는다.</br>
	(영어로는 exhaustive searching 이라고 한다.)

- Hyper Parameter Grid는 한 모델의 hyperparameter 조합을 나타내며,</br>
	Grid Search란 Hyper Parameter Grid에 속하는 모든 parameter 조합을 비교 평가하는 방법을 의미한다.

In [2]:
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier

wine_tree = DecisionTreeClassifier(max_depth=2, random_state=13)
hyper_params = {'max_depth': [2, 4, 7, 10]}

grid_search = GridSearchCV(estimator=wine_tree, param_grid=hyper_params, cv=5)
grid_search.fit(X, y) #--> train_test_split도 알아서 다 해준다.

GridSearchCV(cv=5,
             estimator=DecisionTreeClassifier(max_depth=2, random_state=13),
             param_grid={'max_depth': [2, 4, 7, 10]})

#### --▶ GridSearchCV 결과 확인

In [3]:
import pprint

pp = pprint.PrettyPrinter(indent=4)
pp.pprint(grid_search.cv_results_)

{   'mean_fit_time': array([0.01577735, 0.02425313, 0.04001489, 0.06583714]),
    'mean_score_time': array([0.00565472, 0.00317812, 0.0035841 , 0.00538588]),
    'mean_test_score': array([0.6888005 , 0.66356523, 0.65340854, 0.64401587]),
    'param_max_depth': masked_array(data=[2, 4, 7, 10],
             mask=[False, False, False, False],
       fill_value='?',
            dtype=object),
    'params': [   {'max_depth': 2},
                  {'max_depth': 4},
                  {'max_depth': 7},
                  {'max_depth': 10}],
    'rank_test_score': array([1, 2, 3, 4]),
    'split0_test_score': array([0.55230769, 0.51230769, 0.50846154, 0.51615385]),
    'split1_test_score': array([0.68846154, 0.63153846, 0.60307692, 0.60076923]),
    'split2_test_score': array([0.71439569, 0.72363356, 0.68360277, 0.66743649]),
    'split3_test_score': array([0.73210162, 0.73210162, 0.73672055, 0.71054657]),
    'split4_test_score': array([0.75673595, 0.7182448 , 0.73518091, 0.72517321]),
    'std

#### --▶ 최적의 성능을 가진 모델 확인

In [4]:
grid_search.best_estimator_ #--> Model을 물어본 것~!

DecisionTreeClassifier(max_depth=2, random_state=13)

In [5]:
grid_search.best_score_

0.6888004974240539

In [6]:
grid_search.best_params_

{'max_depth': 2}

## 🔰 Pipeline을 적용한 모델 ➡ GridSearchCV

In [7]:
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler

#--> List형으로 Pipeline으로 작동할 순서와 도구를 넣어준다.
estimators = [
	('scaler', StandardScaler()),
 	('clf', DecisionTreeClassifier())
]

pipe = Pipeline(estimators)

In [8]:
hyper_params = [{'clf__max_depth': [2, 4, 7, 10]}]

GridSearch = GridSearchCV(estimator=pipe, param_grid=hyper_params, cv=5)
GridSearch.fit(X, y)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('clf', DecisionTreeClassifier())]),
             param_grid=[{'clf__max_depth': [2, 4, 7, 10]}])

#### --▶ GridSearchCV 결과 확인

In [11]:
GridSearch.cv_results_

{'mean_fit_time': array([0.02174368, 0.02174125, 0.03631253, 0.05165467]),
 'std_fit_time': array([0.00576223, 0.00324024, 0.00314033, 0.00341314]),
 'mean_score_time': array([0.00539155, 0.00319223, 0.00258918, 0.00299897]),
 'std_score_time': array([0.00241509, 0.00039921, 0.00049308, 0.00063338]),
 'param_clf__max_depth': masked_array(data=[2, 4, 7, 10],
              mask=[False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'clf__max_depth': 2},
  {'clf__max_depth': 4},
  {'clf__max_depth': 7},
  {'clf__max_depth': 10}],
 'split0_test_score': array([0.55230769, 0.51230769, 0.52461538, 0.51692308]),
 'split1_test_score': array([0.68846154, 0.63153846, 0.60846154, 0.61230769]),
 'split2_test_score': array([0.71439569, 0.72363356, 0.67667436, 0.6743649 ]),
 'split3_test_score': array([0.73210162, 0.73210162, 0.73749038, 0.70746728]),
 'split4_test_score': array([0.75673595, 0.7182448 , 0.73518091, 0.72440339]),
 'mean_test_score': array([0.688

#### --▶ 최적의 성능을 가진 모델 확인

In [9]:
GridSearch.best_estimator_

Pipeline(steps=[('scaler', StandardScaler()),
                ('clf', DecisionTreeClassifier(max_depth=2))])

In [10]:
GridSearch.best_score_

0.6888004974240539

#### --▶ 표로 성능 결과 정리

- accuracy의 평균과 표준편차를 살펴본다.

In [13]:
import pandas as pd

score_df = pd.DataFrame(GridSearch.cv_results_)
score_df

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_clf__max_depth,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.021744,0.005762,0.005392,0.002415,2,{'clf__max_depth': 2},0.552308,0.688462,0.714396,0.732102,0.756736,0.688800,0.071799,1
1,0.021741,0.003240,0.003192,0.000399,4,{'clf__max_depth': 4},0.512308,0.631538,0.723634,0.732102,0.718245,0.663565,0.083905,2
2,0.036313,0.003140,0.002589,0.000493,7,{'clf__max_depth': 7},0.524615,0.608462,0.676674,0.737490,0.735181,0.656485,0.081066,3
3,0.051655,0.003413,0.002999,0.000633,10,{'clf__max_depth': 10},0.516923,0.612308,0.674365,0.707467,0.724403,0.647093,0.075525,4


In [14]:

score_df[['params','rank_test_score', 'mean_test_score','std_test_score']]

,params,rank_test_score,mean_test_score,std_test_score
0,{'clf__max_depth': 2},1,0.688800,0.071799
1,{'clf__max_depth': 4},2,0.663565,0.083905
2,{'clf__max_depth': 7},3,0.656485,0.081066
3,{'clf__max_depth': 10},4,0.647093,0.075525
